This function pulls from the Uber Website estimated fares from from a list of zip codes to both MHT and BOS, and returns a dataframe. With the time stamp as the category and hour of day

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import datetime
from tinydb import TinyDB, Query
import urllib3
import xlsxwriter
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.keys import Keys
import datetime

In [1]:
#test zips
#update list of zips
a = '01364,01366,01031,01368,01037,01331,01094,03447,01531,01005,01074,01436,01468,03452,01068,01475,01438,01452,01441,01440,03461,01543,03458,01612,01430,01473,01541,01522,03071,03084,01520,01431,01420,01583,01564,03086,01453,03048,01474,01462,01505,01469,01510,01561,03033,03055,01523,01464,01472,01503,01467,01434,03031,01740,01463,03049,01451,01450,01470,01471,01432,01749,03063,01719,03054,01775,01827,03062,01460,03064,03061,03060,01754,03052,01886,01720,01879,01718,01776,03051,03053,01863,01742,01741,01824,01854,01851,03076,01773,01853,01826,01850,03087,01852,01862,03041,01731,03038,01730,01822,01865,02451,01821,02421,01876,01866,03079,03073,02420,01803,02479,01805,01844,05501,02476,03841,01887,01810,01842,01841,02474,01843,03811,01840,01801,01813,01815,01888,01890,02475,01812,01899,01832,02156,03819,03826,02155,01867,02180,03865,01845,01864,01835,01889,01831,01880,03859,01885,03848,01830,02148,02176,03858,01940,01921,01949,03861,01906,01860,01834,03833,01937,01905,01833,03827,01960,01904,01985,03856,03857,01913,01923,01910,01903,01901,01983,01902,03823,01961,03824,01922,01970,01907,03885,01971,01969,03844,01950,01984,03874,01945,01952,01982,01915,01936,03843,03840,01951,01938,03842,03862,01965,03801,03903,01929,03871,01944,03802,03803,03804,03870,03904,03854,03905,01930,03909,01931,03911,01966'

In [2]:
def pull_uber_fares(list_zip_codes):
    '''the only input should be a list of zip codes as a string for your selection'''
    '''returns a dataframe'''
    
    #import necessary dependencies
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium import webdriver
    import datetime
    from tinydb import TinyDB, Query
    import urllib3
    import xlsxwriter
    from selenium import webdriver
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.support.ui import Select

    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import time
    from selenium.webdriver.common.keys import Keys
    import datetime
    
    #pull apart string zip codes
    zip_codes = []
    for b in [b for b in list_zip_codes.split(',')]:
        zip_codes.append(b)
    #create driver    
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #lets try uber first
    uber = 'https://www.uber.com/us/en/price-estimate/'
    #get the website
    driver.get(uber)

    airports = ['Manchester-Boston Regional Airport, 1 Airport Rd, Manchester, NH',
               'Boston Logan International Airport, 1 Harborside Dr, Boston, MA']

    #these are lists of lists len3
    MHT = []
    BOS = []
    airport_price_lists = [MHT,BOS]

    uber_Tier1_prices = []
    uber_Tier2_prices = []
    uber_Tier3_prices = []

    #pass over both aiports
    for h in range(0, len(airports)):
        #find element id
        for i in range(0,len(zip_codes)):
            driver.implicitly_wait(4)
            starting = driver.find_element_by_name('pickup')
            time.sleep(1)
            starting.clear()
            starting.send_keys(zip_codes[i])
            time.sleep(1)
            starting.send_keys(Keys.ENTER)

            starting = driver.find_element_by_name('destination')
            time.sleep(1)
            starting.clear()
            starting.send_keys(airports[h])
            time.sleep(1)
            starting.send_keys(Keys.RETURN)
            time.sleep(1)

            #pull values from website
            #First Tier
            try:
                element = driver.find_element_by_xpath('//*[@id="main"]/div[2]/div/div/div/div[1]/div[2]/div[1]/div[1]/div[1]/div/span')
                uber_Tier1_prices.append(element.text)
            except:
                uber_Tier1_prices.append(np.nan)
            
            #Second Tier
            try:        
                element = driver.find_element_by_xpath('//*[@id="main"]/div[2]/div/div/div/div[1]/div[2]/div[1]/div[2]/div[1]/div/span')
                uber_Tier2_prices.append(element.text)
            except:
                uber_Tier2_prices.append(np.nan)
            #Third Tier
            try:
                element = driver.find_element_by_xpath('//*[@id="main"]/div[2]/div/div/div/div[1]/div[2]/div[1]/div[3]/div[1]/div/span')
                uber_Tier3_prices.append(element.text)
            except:
                uber_Tier3_prices.append(np.nan)
        
        #dump into right lists
        airport_price_lists[h].append(uber_Tier1_prices)
        airport_price_lists[h].append(uber_Tier2_prices)
        airport_price_lists[h].append(uber_Tier3_prices)

        #reassign ubers_lists
        uber_Tier1_prices = []
        uber_Tier2_prices = []
        uber_Tier3_prices = []
        #done with loops
        
        #create zip_codes df
        zip_codes_df_BOS = pd.DataFrame(zip_codes,columns=["Zip_Codes"])
        zip_codes_df_MHT = pd.DataFrame(zip_codes,columns=["Zip_Codes"])
        
    
    #put back in the right frames, start with MHT
    uber_col_names = ['Tier1','Tier2','Tier3']
    for i in range(0,len(airport_price_lists[0])):
       #add to dataframe
        zip_codes_df_MHT[uber_col_names[i]] = airport_price_lists[0][i]
    
    
    #put back into BOSTON
    uber_col_names = ['Tier1','Tier2','Tier3']
    for i in range(0,len(airport_price_lists[0])):
       #add to dataframe
        zip_codes_df_BOS[uber_col_names[i]] = airport_price_lists[1][i]
    
    #add in Desitnation airpots
    zip_codes_df_MHT['Destination'] = ['MHT' for foo in range(len(zip_codes))]
    zip_codes_df_BOS['Destination'] = ['BOS' for foo in range(len(zip_codes))]
    
    
    #stack frames
    final_frame = pd.concat([zip_codes_df_MHT,zip_codes_df_BOS])
    #add in compnay name
    final_frame['Company'] = ['Uber' for foo in range(final_frame.shape[0])]
    #get day of week
    day_week = datetime.datetime.today().weekday()
    #get hour of day
    hour = datetime.datetime.now().hour
    final_frame['Day'] = [day_week for foo in range(final_frame.shape[0])]
    final_frame['Hour'] = [hour for foo in range(final_frame.shape[0])]
    
    #add in company name
    final_frame['Company'] = ['Uber' for foo in range(final_frame.shape[0])]
    
    #melt
    final_frame = pd.melt(final_frame,id_vars=['Zip_Codes','Destination','Company','Day','Hour'],
       value_vars=['Tier1','Tier2','Tier3'],var_name='Tier',value_name='Fare')
    
    return(final_frame)
    
    
    

In [3]:
b = pull_uber_fares(list_zip_codes=a)

Trying to download new driver from http://chromedriver.storage.googleapis.com/79.0.3945.36/chromedriver_mac64.zip
Unpack archive /Users/janmichaelaustria/.wdm/drivers/chromedriver/79.0.3945.36/mac64/chromedriver.zip


In [8]:
b.to_csv('Uber_Fares_Friday_AM.csv')

WORKS!! YAY!!

This code pulls travel time and distances from mapquest, again the only input function should be a string of zip codes

In [30]:
def pull_distance_time(list_zip_codes):
    '''the only input should be a list of zip codes as a string for your selection'''
    '''returns a dataframe'''
    
    #import necessary dependencies
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium import webdriver
    import datetime
    from tinydb import TinyDB, Query
    import urllib3
    import xlsxwriter
    from selenium import webdriver
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.support.ui import Select

    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import time
    from selenium.webdriver.common.keys import Keys
    import datetime
    
    #pull apart string zip codes
    zip_codes = []
    for b in [b for b in list_zip_codes.split(',')]:
        zip_codes.append(b)
    #create driver    
    driver = webdriver.Chrome(ChromeDriverManager().install())
    map_quest = 'https://www.mapquest.com/directions'
    #get the website
    driver.get(map_quest)

    airports = ['Manchester-Boston Regional Airport, 1 Airport Rd, Manchester, NH',
               'General Edward Lawrence Logan International Airport (BOS)']
    
    #these are lists of lists len2
    MHT = []
    BOS = []
    map_quest_lists = [MHT,BOS]
    
    #init lists to store miles and distance
    list_travel_times = []
    list_distances = []
    
    #pass over both aiports
    for h in range(0, len(airports)):
        #locate destination box
        driver.implicitly_wait(3)
        starting = driver.find_element_by_xpath('//*[@id="primaryPanel"]/div[5]/div[3]/div/div/div[2]/div[1]/form/div[1]/div[1]/ul/li[2]/directions-route-stop/div/dnd-nodrag/ng-transclude/div/div[1]/div/input')
        starting.clear()
        #input airport
        starting.send_keys(airports[h])
        time.sleep(1)
        starting.send_keys(Keys.DOWN)
        time.sleep(1)
        starting.send_keys(Keys.RETURN)
        time.sleep(1)
        #find element ids
        for i in range(0, len(zip_codes)):
            #find input
            starting = driver.find_element_by_xpath('//*[@id="primaryPanel"]/div[5]/div[3]/div/div/div[2]/div[1]/form/div[1]/div[1]/ul/li[1]/directions-route-stop/div/dnd-nodrag/ng-transclude/div/div[1]/div/input')
            time.sleep(1)
            starting.clear()
            starting.send_keys(zip_codes[i])
            time.sleep(1)
            starting.send_keys(Keys.DOWN)
            time.sleep(1)
            starting.send_keys(Keys.ENTER)
            time.sleep(1)
                
            try:
                element = driver.find_element_by_xpath('//*[@id="primaryPanel"]/div[5]/div[3]/div/div/div[2]/div[1]/form/div[2]/div/div/div/route-info/div/div[2]/div[2]/span[1]')
                list_travel_times.append(element.text)
            except:
                list_travel_times.appendend(np.nan)
            try:      
                element = driver.find_element_by_xpath('//*[@id="primaryPanel"]/div[5]/div[3]/div/div/div[2]/div[1]/form/div[2]/div/div/div/route-info/div/div[2]/div[2]/div')
                list_distances.append(element.text)
            except:
                list_distances.append(np.nan)
                
        #dump into right lists
        map_quest_lists[h].append(list_travel_times)
        map_quest_lists[h].append(list_distances)
        
        #reassign ubers_lists
        list_travel_times = []
        list_distances = []
        #done with loops
        
    #put into frames now
    zip_codes_df_BOS = pd.DataFrame(zip_codes,columns=["Zip_Codes"])
    zip_codes_df_MHT = pd.DataFrame(zip_codes,columns=["Zip_Codes"])
    
    #put back in the right frames, MHT
    col_names = ['Time','Distance']
    for i in range(0,len(map_quest_lists[0])):
       #add to dataframe
        zip_codes_df_MHT[col_names[i]] = map_quest_lists[0][i]
        
    #put back in the right frames, BOS
    col_names = ['Time','Distance']
    for i in range(0,len(map_quest_lists[0])):
       #add to dataframe
        zip_codes_df_BOS[col_names[i]] = map_quest_lists[1][i]
        
    #add in Desitnation airpots
    zip_codes_df_MHT['Destination'] = ['MHT' for foo in range(len(zip_codes))]
    zip_codes_df_BOS['Destination'] = ['BOS' for foo in range(len(zip_codes))]
    
    #stack frames
    final_frame = pd.concat([zip_codes_df_MHT,zip_codes_df_BOS])
    
    #get day of week
    day_week = datetime.datetime.today().weekday()
    #get hour of day
    hour = datetime.datetime.now().hour
    final_frame['Day'] = [day_week for foo in range(final_frame.shape[0])]
    final_frame['Hour'] = [hour for foo in range(final_frame.shape[0])]
    
    return(final_frame)

In [31]:
c = pull_distance_time(list_zip_codes=a)


Looking for [chromedriver 79.0.3945.36 mac64] driver in cache 
File found in cache by path [/Users/janmichaelaustria/.wdm/drivers/chromedriver/79.0.3945.36/mac64/chromedriver]


AttributeError: 'list' object has no attribute 'appendend'

In [17]:
a

'01364,01366,01031,01368,01037,01331,01094,03447,01531,01005,01074,01436,01468,03452,01068,01475,01438,01452,01441,01440,03461,01543,03458,01612,01430,01473,01541,01522,03071,03084,01520,01431,01420,01583,01564,03086,01453,03048,01474,01462,01505,01469,01510,01561,03033,03055,01523,01464,01472,01503,01467,01434,03031,01740,01463,03049,01451,01450,01470,01471,01432,01749,03063,01719,03054,01775,01827,03062,01460,03064,03061,03060,01754,03052,01886,01720,01879,01718,01776,03051,03053,01863,01742,01741,01824,01854,01851,03076,01773,01853,01826,01850,03087,01852,01862,03041,01731,03038,01730,01822,01865,02451,01821,02421,01876,01866,03079,03073,02420,01803,02479,01805,01844,05501,02476,03841,01887,01810,01842,01841,02474,01843,03811,01840,01801,01813,01815,01888,01890,02475,01812,01899,01832,02156,03819,03826,02155,01867,02180,03865,01845,01864,01835,01889,01831,01880,03859,01885,03848,01830,02148,02176,03858,01940,01921,01949,03861,01906,01860,01834,03833,01937,01905,01833,03827,01960,019

In [148]:
d

,Zip_Codes,Time,Distance
0,01430,1hr 22min,65.8mi
1,03071,1hr 34min,69.3mi
2,01431,1hr 23min,57.5mi
3,01420,1hr 6min,54.6mi


In [92]:
h

1